In [5]:
import os
from dotenv import load_dotenv
from PyPDF2 import PdfReader

load_dotenv()

True

In [6]:
from astrapy import DataAPIClient

application_token = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
astra_db_id = os.getenv("ASTRA_DB_ID")
astra_db_api_endpoint = os.getenv("ASTRA_DB_API_ENDPOINT")
my_client = DataAPIClient(application_token)

my_db = my_client.get_database(token=application_token, api_endpoint=str(astra_db_api_endpoint))

print(f"Connected to Astra DB: {my_db.list_collection_names()}")

pdfreader = PdfReader("data/Ch 01 SQC 01 SA 220.pdf")
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

Connected to Astra DB: []


In [7]:

from langchain.text_splitter import CharacterTextSplitter
import os
from langchain_community.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
embedding = OpenAIEmbeddings(model="text-embedding-3-large",openai_api_key=OPENAI_API_KEY)

# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 600,
    chunk_overlap  = 150,
    length_function = len,
)
chunks = text_splitter.split_text(raw_text)
chunks[:50]

C:\Users\satpu\AppData\Local\Temp\ipykernel_4976\3435097645.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model="text-embedding-3-large",openai_api_key=OPENAI_API_KEY)
Created a chunk of size 6203, which is longer than the specified 600


["LetChapterSaSQC.020IntroductionAuditQuality.EtoSoc.060⑦AuditQuality-Walk-systematicallyasper·LegalRequirement(LR)·RegulatoryReg(RR)⊥Report·Raf&Te c h n i c a l(PTS)standards->Appropriateingivencircumstances·Reliable·PlopelForm③2standardstoensurequality--SQC·SA220①SQC1-FormaintainingqualityofallAssurance&RelatedEngagementsJuFocuson6ElementsLE-AnyaHeman⊥-LeadershipResponsibilitiesE-EthicalRequirement&IndependenceA-Acceptance&ContinuanceH->HumanResourceE-EngagementPerformancem->Monitoring↓Policies&procedurestoimplementabove6elementsiscollectivelycalledSystemofQualityofContral↓ItisfarallaswanceIrelatedassignmentsatFirmlevel,henceitiscalledFirmlevelsystemofQualityCentral⑪SA220-AuditofHistoricalInformationisthemostimportantalignmentToenwequalityofadaWehaveSazzoInfo)↓ImplementationofFirmlevelSystemofQualityCortealatEngagementlevelSac.080LeadershipResponsibilitiesfarSystemofqualitycontrol①Culture-Firmspolicies&proceduresshouldensurethatfirm'scultureshouldgivehighestimportancetoestablishing&i

In [8]:
# Support for dataset retrieval with Hugging Face
from datasets import load_dataset
# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

application_token = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")
astra_db_api_endpoint = os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_REGION=os.getenv("ASTRA_DB_REGION")

cassio.init(token=application_token, database_id=ASTRA_DB_ID)
embedding = OpenAIEmbeddings(model="text-embedding-3-large",openai_api_key=OPENAI_API_KEY)

astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="pdfnotes",
    session=None,
    keyspace=None
)

# astra_vector_store.add_texts(chunks[:50])

# print("Inserted %i headlines." % len(chunks[:50]))

# astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)


In [ ]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
import tiktoken
#llm = OpenAI(openai_api_key=OPENAI_API_KEY)
llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k",  # <-- Important to avoid 4097 token limit
    openai_api_key=OPENAI_API_KEY
)

query_text = "Explain me about the key consideration while determining Competence & Capability"

answer = astra_vector_index.query(query_text, llm=llm).strip()
print("ANSWER: \"%s\"\n" % answer)

ANSWER: "The key considerations while determining Competence & Capability in the context of auditing include the following factors:

1. Regulations: Ensuring that the firm has adequate knowledge of applicable legal requirements, regulatory standards, and professional technical standards for the specific engagement.
  
2. Deadlines: Ability to complete the engagement within the given deadline while maintaining quality as per the client's expectations.
   
3. Experts: Availability of experts for handling complex and technical issues that may arise during the audit process.
  
4. Resources: Ensuring that there are sufficient resources in terms of knowledge, skills, time, team, and equipment to carry out the audit effectively.
  
5. Knowledge: Having industry-specific knowledge and understanding of the applicable financial reporting frameworks.
  
These factors are crucial in evaluating the competence and capability of the audit team to perform the engagement effectively and deliver high-q